In [1]:
import numpy as np
import sympy
import cirq
import tensorflow as tf
import tensorflow_quantum as tfq

n_qubits = 3
qubits = cirq.GridQubit.rect(1, n_qubits)
observable = [1.*cirq.X.on(q) for q in qubits] # -J \sum_{i} Z_i Z_{i+1} - g \sum_i X_i    when g>>J
for q in range(len(qubits)):
    observable.append(0.1*cirq.Z.on(qubits[q])*cirq.Z.on(qubits[(q+1)%len(qubits)]))

In [2]:
observable

[X((0, 0)),
 X((0, 1)),
 X((0, 2)),
 (0.1+0j)*Z((0, 0))*Z((0, 1)),
 (0.1+0j)*Z((0, 1))*Z((0, 2)),
 (0.1+0j)*Z((0, 0))*Z((0, 2))]

In [3]:
circuit = cirq.Circuit()
symbols=[]
for k in range(3):
    sym = sympy.Symbol("a"+str(k))
    symbols.append(sym)
    circuit.append(cirq.ry(sym).on(qubits[k]))

In [4]:
circuit_input = tf.keras.Input(shape=(), dtype=tf.string)
output = tfq.layers.Expectation()(
        circuit_input,
        symbol_names=symbols,
        operators=tfq.convert_to_tensor([observable]),
        initializer=tf.keras.initializers.RandomNormal()) #notice this is not strictly necessary.

output = tf.math.reduce_sum(output, axis=-1, keepdims=True)
model = tf.keras.Model(inputs=circuit_input, outputs=output)
adam = tf.keras.optimizers.Adam(learning_rate=0.1)
model.compile(optimizer=adam, loss='mse')

In [ ]:
indexed_cnots = {}
count = 0
for control in range(n_qubits):
    for target in range(n_qubits):
        if control != target:
            indexed_cnots[str(count)] = [control, target]
            count += 1
number_of_cnots = len(indexed_cnots)
#int(np.math.factorial(n_qubits)/np.math.factorial(n_qubits -2))

# Create one_hot alphabet
alphabet_gates = [cirq.CNOT, cirq.ry, cirq.rx(-np.pi/2), cirq.I]
alphabet = []

alphabet_length = number_of_cnots + (len(alphabet_gates)-1)*n_qubits
for ind, k in enumerate(range(number_of_cnots + (len(alphabet_gates)-1)*n_qubits)): #5 accounts for 2 CNOTS and 3 other ops
    one_hot_gate = [-1]*alphabet_length
    one_hot_gate[ind] = 1
    alphabet.append(one_hot_gate)


In [ ]:
len(alphabet)

In [ ]:
alphabet_gates

In [ ]:
class VAnsatzSmart(CirqSmartSolver):
    def __init__(self, n_qubits, observable_name, target_reward, trajectory):
        """
        takes as input a list with actions, each action being an integer between 0 and len(self.alphabet).
        """
        super(VAnsatzSmart, self).__init__(n_qubits, observable_name)

        ws = [self.alphabet[int(g)] for g in trajectory]

        ws = self.check_and_recheck(ws)
        ws = self.detect_u3_and_reduce(ws)

        #### this is as a checker, to be removed later on! ###
        # val = np.sum(np.diag(np.array(self.alphabet)))
        # if val != len(self.alphabet) :
        #     print(val)
        #     print(len(self.alphabet))
        #     print("WARNING!!!!")

        circuit = []
        params = []
        for g in ws:
            circuit, params = self.append_to_circuit(g,circuit, params)

        self.circuit = cirq.Circuit(circuit)

        #### check if there are qubits not called in the circuit  and add identity.
        #### (otherwise a problem with observable appears)

        effective_qubits = list(self.circuit.all_qubits())
        for k in self.qubits:
            if k not in effective_qubits:
                self.circuit.append(cirq.I.on(k))
        self.symbols = params

    def get_state(self, qubits, params=None):
        if params is None:
            return self.circuit
        resolver = {k: v for k, v in zip(self.symbols, params)}
        return cirq.resolve_parameters(self.circuit, resolver)